## Data import

In [ ]:
! pip install gdown
from IPython import display
display.clear_output()

In [ ]:
! gdown 14isqC-JCu9_uztKXftKT-VZhYN_rwHrx
! gdown 1HDHm55EZXpHDX_43tB3yt_Qqw0byd4ZT
! gdown 1jNgjGidE4FFXPAmcxtmjJbFJeDbB1BZw
! gdown 1Z-QfsEes19cAdjqF7WlDCu8Dw-FnyBBd
! gdown 18xExKposyAMHBx7kuWYF2T1Pl2oVMlPU

In [ ]:
! unzip -q ByteTrack.zip
! unzip -q our_dataset_emb.zip

## Install dependencies

In [ ]:
! pip install face_recognition

from IPython import display
display.clear_output()

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
%cd {HOME}/ByteTrack
! wget https://raw.githubusercontent.com/ifzhang/ByteTrack/main/README.md
! bash install.sh
from IPython import display
display.clear_output()

In [ ]:
from yolox.tracker.byte_tracker import BYTETracker
from tracker import BYTETrackerArgs, detections2boxes, match_detections_with_tracks
from tqdm import tqdm
from ultralytics import YOLO
import threading
import  supervision as sv 
import numpy as np
import cv2
import face_recognition

## Model declear

In [ ]:
# MODEL
MODEL = "yolov8x-pose.pt"
model = YOLO(MODEL)
# embedding_model = DeepFace.build_model('Dlib')

# Tracker
byte_tracker = BYTETracker(BYTETrackerArgs())

In [ ]:
SOURCE_VIDEO_PATH = "/content/main_video.mp4"
TARGET_VIDEO_PATH = "/content/main_video_output.mp4"

# SOURCE_VIDEO_PATH = "/content/final_video.mp4"
# TARGET_VIDEO_PATH = "/content/final_video_output.mp4"

## Person identification

### Load known face encoding

In [ ]:
from glob import glob
import platform

In [ ]:
def get_known_face_embedding(path):
    """
    path: dirpath
    full path will be: {path}/*/*.npy
    """
    # emb_paths = glob(f"{path}/*/*.npy")
    emb_paths = glob(f"{path}/*/*.npy")
    if platform.system() == "Windows":
        known_face_ids = [x.split("\\")[-2] for x in emb_paths]
    else:
        known_face_ids = [x.split("/")[-2] for x in emb_paths]
    known_embedding = []
    for emb_path in emb_paths:
        known_embedding.append(np.load(emb_path))
    return np.array(known_embedding), np.array(known_face_ids)

In [ ]:
known_face_embedding, known_face_ids = get_known_face_embedding("/content/our_dataset_emb")
known_face_embedding.shape, known_face_ids.shape

((1761, 128), (1761,))

### unknowing face encoding

In [ ]:
def get_face_locations(boxes, all_keypoints):
    """
    results: yolo model detection results
    results = model(img)
    """
    # boxes = np.array(results[0].boxes.data.tolist())
    # all_keypoints = np.array(results[0].keypoints.data.tolist())
    all_min_x = all_keypoints[:,:5,0].min(axis=1)
    all_max_x = all_keypoints[:,:5,0].max(axis=1)
    all_mid_x = all_keypoints[:,0,0]

    all_min_y = all_keypoints[:,:5,1].min(axis=1)
    all_max_y = all_keypoints[:,:5,1].max(axis=1)
    all_mid_y = all_keypoints[:,0,1]
    distance = np.max(
        np.absolute(
            [
                all_mid_x - all_min_x,
                all_max_x - all_mid_x,
                all_mid_y - all_min_y,
                all_max_y - all_mid_y
            ]
        ), axis=0
    )

    y1 = all_mid_y - distance
    x2 = all_max_x
    y2 = all_mid_y + distance
    x1 = all_min_x

    face_locations = np.array([y1, x2, y2, x1], dtype=np.int32).T
    return face_locations


In [ ]:
def is_face_available(all_keypoints):
    """
    results: yolo model detection results
    results = model(img)
    """
    mask = all_keypoints[:,:3,-1]>.9
    count_mask = np.sum(mask, axis=1)
    return count_mask == 3


## Track_recoder

In [ ]:
from datetime import datetime
import pandas as pd
import threading
import time
import numpy as np

class TrackRecoder:
    def __init__(self):
        self.Identified_ids = []
        self.Identified_ids_link_person_ids = {}
        self.NonIdentified_track_records = {}
        self.Identified_track_records = []
        self.create_record_file()

    def get_time(self):
        t = datetime.now()
        return f"{t.year:04}-{t.month:02}-{t.day:02}-{t.hour:02}-{t.minute:02}"

    def create_record_file(self):
        t = self.get_time()
        self.file_path  = f"person_track_{t}.csv"
        with open(self.file_path, "w") as file:
            file.write("person_id,x,y,time")

    def update_track(self, records):
        """
        records: list of records
        every record
        record[0]: track id
        record[1]: Identified person id
        recode[2]: point x
        recode[3]: point y
        """
        t = self.get_time()
        
        for record in records:
            id, person_id, x, y = record

            if id not in self.Identified_ids and person_id is not None:
                self.Identified_ids.append(id)
                self.Identified_ids_link_person_ids[id] = person_id
                if id in self.NonIdentified_track_records:
                    id_recodes = np.array(self.NonIdentified_track_records[id])
                    self.NonIdentified_track_records.pop(id)
                    id_recodes[id_recodes==None] = person_id
                    self.Identified_track_records += id_recodes.tolist()
                    del(id_recodes)



            if id in self.Identified_ids:
                person_id = self.Identified_ids_link_person_ids[id]
                self.Identified_track_records.append([person_id, x, y, t])
            elif person_id is None:
                if id not in self.NonIdentified_track_records:
                    self.NonIdentified_track_records[id] = []
                self.NonIdentified_track_records[id].append([person_id, x, y, t])


            


    def autoUpdate(self):
        columns = ["person_id", 'x', 'y', "time"]
        main_df = pd.read_csv(self.file_path)
        temp_df = pd.DataFrame(self.Identified_track_records, columns=columns)
        self.Identified_track_records = []
        main_df = pd.concat([main_df, temp_df], ignore_index=True)
        main_df.to_csv(self.file_path, index=False)
        del(main_df)
        del(temp_df)

    def check_NonIdentified_track_records(self):

        t = datetime.now()
        now_t = t.hour*60 + t.minute
        record_keys = list(self.NonIdentified_track_records.keys())  # create a list of keys
        keys_to_remove = []
        for record in record_keys:
            t = self.NonIdentified_track_records[record][-1][-1].split('-')[-2:]
            t = int(t[0])*60 + int(t[1])
            if abs(now_t - t) > 5:
                keys_to_remove.append(record)  # add key to list of keys to remove

        # remove the keys outside the loop
        for key in keys_to_remove:
            self.NonIdentified_track_records.pop(key)

    def start_timer(self, interval):
        self.interval = interval
        self.timer = threading.Timer(self.interval, self.start_timer, args=[self.interval])
        self.timer.start()
        self.autoUpdate()
        self.check_NonIdentified_track_records()

    def stop_timer(self):
        self.timer.cancel()


### Face verification

In [ ]:
def face_distance(face_encodings, face_to_compare):
    """
    face_encodings.shape = (n, 128)
    face_to_compare.shape = (k, 128)
    return_value = np.linalg.norm(face_encodings - face_to_compare, axis=1)
    return_value.shape = (k,)
    but i want to return shape = (n,k)
    """
    if len(face_encodings) == 0:
        return np.empty((0))
    return np.linalg.norm(face_encodings[:, np.newaxis, :] - face_to_compare[np.newaxis, :, :], axis=-1)

In [ ]:
def findCosineDistance(source_representation, test_representation):
    """
    source_representation.shape = (n,128)
    test_representation.shape = (k,128)
    return shape = (n,k)
    """
    a = np.matmul(source_representation, test_representation.T)
    b = np.sum(np.multiply(source_representation, source_representation), axis=1, keepdims=True)
    c = np.sum(np.multiply(test_representation, test_representation), axis=1, keepdims=True)
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

In [ ]:
thresholds = {
        "Dlib": {"cosine": 0.07, "linalg": 0.5}
    }

In [ ]:
def face_verification(
    known_face_embedding, test_embedding, known_face_ids, threshold=0.6, distance_type='linalg'):
    """
    known_face_embedding.shape = (n,128)
    test_embedding.shape = (k,128)
    known_face_ids.shape = (n,)
    """
    known_face_ids = np.array(known_face_ids)
    if distance_type == "linalg":
        face_distances = face_distance(known_face_embedding, test_embedding)
    else:
        face_distances = findCosineDistance(known_face_embedding, test_embedding)
    faceDis = face_distances.min(axis=0)
    arg_min = face_distances.argmin(axis=0)
    ids = known_face_ids[arg_min]
    ids = np.where(faceDis<=threshold, ids, None)
    return ids



In [ ]:
def get_person_ids(
    face_locations, face_available, boxes_conf,
    known_face_embedding, known_face_ids,
    trackId, recorder, threshold=0.6, distance_type='linalg'
):
    Identified_ids = recorder.Identified_ids
    # trackId = detections.tracker_id
    isin_trackId = np.isin(trackId, Identified_ids, invert=True)
    is_boxes = boxes_conf > .4
    is_face_check = isin_trackId & face_available & is_boxes

    embeddings = []


    """_________________________________________"""
    check_location = []
    for i in range(trackId.shape[0]):
        if is_face_check[i]:
            check_location.append(face_locations[i])
    check_location = np.array(check_location)
    embeddings = face_recognition.face_encodings(frame, check_location)
    """_________________________________________"""
    embeddings = np.array(embeddings)
    # print(embeddings.shape)
    if embeddings.shape[0] > 0:
        ids = face_verification(known_face_embedding, embeddings, known_face_ids,
                            threshold=threshold, distance_type='linalg')

    lables = []
    j = 0
    for i in range(face_locations.shape[0]):
        try:
            if not isin_trackId[i]:
                lables.append(recorder.Identified_ids_link_person_ids[trackId[i]])
            elif is_face_check[i]:
                lables.append(ids[j])
                j+=1
            else:
                lables.append(None)
        except Exception as e:
            raise Exception(f"{e}")
    


    """___________________duplicate value check____________________"""
    np_labels = np.array(lables)
    valid_indices = np.where(np_labels != None)[0]
    filtered_lables = np_labels[valid_indices]
    if len(filtered_lables)>0:
      
      print(filtered_lables)
      if np.unique(filtered_lables, return_counts=True)[1].max() > 1:
          labels_dict = {}
          for i in range(face_locations.shape[0]):
              if lables[i]:
                if lables[i] not in labels_dict:
                    labels_dict[lables[i]] = []
                labels_dict[lables[i]].append(trackId[i])
          labels_dict = {k: sorted(v) for k, v in labels_dict.items()}
          for k, v in labels_dict.items():
              for duplicate_id in v[:-1]:
                  if duplicate_id in recorder.Identified_ids:
                      recorder.Identified_ids.remove(duplicate_id)
                      indices = np.where(trackId == duplicate_id)[0][0]
                      lables[indices]=None

    """___________________duplicate value check____________________"""


    return lables

### Utilits

In [ ]:
def get_records(detections, ids, all_keypoints):
    a = detections.tracker_id.copy()
    b = np.array(ids, dtype=object)
    c =  all_keypoints[:,0,:2].astype(np.int32).copy() # shape (6,2)

    # Reshape `a` and `b` to column vectors
    a = np.reshape(a, (-1, 1))
    b = np.reshape(b, (-1, 1))
    # Concatenate `a`, `b`, and `c` along axis 1
    return np.concatenate((a, b, c), axis=1)

# main function

In [ ]:
name_df = pd.read_csv("/content/our_dataset_emb/info.csv")
name_df['Roll'] = name_df['Roll'].astype(str)
name_dict = name_df.set_index('Roll')['Name'].to_dict()
name_dict

In [ ]:
attendace_dict = {}
attendace_count = 0

In [ ]:
# Tracker
import time

byte_tracker = BYTETracker(BYTETrackerArgs())


# # Known face encoding
# personIdentification = PersonIdentityDetection()
# known_encodings, known_face_ids  = personIdentification.get_knownFaceEncoding_label("data/known_face")


# Record
recorder = TrackRecoder()
recorder.start_timer(60)

# SOURCE_VIDEO_PATH = "data/input/kitiParty.mp4"
# TARGET_VIDEO_PATH = "data/output/kitiParty.mp4"



generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

# create instance of BoxAnnotator
annotator = sv.BoxAnnotator(thickness=2, text_thickness=1, text_scale=1)
# create VideoInfo instance
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)


start_time = time.perf_counter ()
fps = 8
fps_count = 0
# create instance of VideoWriter
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as writer:
    for frame in tqdm(generator, total=video_info.total_frames):
        # get boxes
        results = model(frame)[0]
        detections = sv.Detections.from_yolov8(results)
        if detections.class_id.shape[0]!=0:
            tracks = byte_tracker.update(
                output_results=detections2boxes(detections=detections),
                img_info=frame.shape,
                img_size=frame.shape
            )

            tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
            detections.tracker_id = np.array(tracker_id)



            # filtering out detections without trackers
            mask = np.array([tracker_id is not None  for tracker_id in detections.tracker_id], dtype=bool)
            detections.filter(mask=mask, inplace=True)
            detections.xyxy = detections.xyxy.astype('int32')


            boxes = results.boxes.xyxy.cpu().numpy()[mask==True]
            boxes_conf = results.boxes.conf.cpu().numpy()[mask==True]
            all_keypoints = results.keypoints.cpu().numpy()[mask==True]
            face_locations = get_face_locations(boxes, all_keypoints)
            face_available = is_face_available(all_keypoints)
            


            ids = get_person_ids(
                face_locations, face_available, boxes_conf,
                known_face_embedding, known_face_ids,
                detections.tracker_id, recorder, threshold=0.4, distance_type='linalg'
            )
            records = get_records(detections, ids, all_keypoints)
            recorder.update_track(records)
            
            

            # draw frame
            fps_count += 1
            end_time = time.perf_counter ()
            if end_time - start_time >= 1.:
                start_time = time.perf_counter ()
                fps = fps_count
                fps_count = 0


            cv2.putText(frame, f"FPS: {fps}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            # frame = annotator.annotate(scene=frame, detections=detections, labels=ids)
            "________________________________________"
            # new add
            labels = [None]*len(ids)
            for i, id in enumerate(ids):
                if id:
                    labels[i] = f"{id} {name_dict[id]}"

            frame = annotator.annotate(scene=frame, detections=detections, labels=labels)
            "________________________________________"


            "___________________Start Take Attendance______________________"
            # attendace_count += 1
            # for label in ids:
            #     if label:
            #         if label not in attendace_dict.keys():
            #             attendace_dict[label] = attendace_count
            
            "___________________End Take Attendance________________________"


        # write frame
        writer.write_frame(frame)
recorder.stop_timer()

print(attendace_dict)